<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('wordnet')
import re
import unicodedata
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
project_path = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
%pip install https://github.com/Xtra-Computing/thundersvm/releases/download/v0.3.4/thundersvm_cuda10.1-0.3.4-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 522kB 1.2MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# remove unwanted chars
# convert to lowercase
# remove unwanted spaces
# remove stop words
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 

def preprocess_text(corpus, text_lower_case=True, 
                      special_char_removal=True, stopword_removal=True, remove_digits=False):    
    normalized_text = []
    # normalize each document in the corpus
    for doc in corpus:
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits) 

        if stopword_removal:
            doc = remove_stopwords(doc)

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)            
        normalized_text.append(doc)
        
    return normalized_text

def remove_special_characters(text, remove_digits=False):
    #Using regex
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_stopwords(text):  
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]   
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)                 
    return ' '.join(filtered_sentence)

In [1]:
from sklearn.model_selection import train_test_split

squad_df = pd.read_csv(project_path+'squad_data_final.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)
squad_df.tail(5)

NameError: ignored

In [0]:
squad_df[squad_df['id'] == '5730f1c3e6313a140071cad6']

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible,clean_context,clean_question,clean_answer


In [0]:
squad_df_cleaned = squad_df.head(35000)

In [0]:
Xcontext_train, Xcontext_test, yquestion_train, yquestion_test = train_test_split(squad_df_cleaned['clean_context'], squad_df_cleaned['clean_question'], test_size=0.33, random_state=42)
Xquestion_train, Xquestion_test, yanswer_train, yanswer_test = train_test_split(squad_df_cleaned['clean_question'], squad_df_cleaned['clean_answer'], test_size=0.33, random_state=42)

In [10]:
print(Xcontext_train.shape)
print(Xcontext_test.shape)
print(yquestion_train.shape)
print(yquestion_test.shape)

(23450,)
(11550,)
(23450,)
(11550,)


In [11]:
# Define TF-ID Venctorize and Label encoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

xvectorizer = TfidfVectorizer(max_features=5000)
xvectorizer.fit(squad_df_cleaned['clean_context'] + " " + squad_df_cleaned['clean_question'])
ylblencoder = LabelEncoder()
# # generate label encoder for both question and clean answer
ylblencoder.fit(squad_df_cleaned['clean_question'].append(squad_df_cleaned['clean_answer']).reset_index(drop=True).astype(str))

LabelEncoder()

In [12]:
print(len(ylblencoder.classes_))
len(xvectorizer.vocabulary_)

53819


5000

In [0]:
# Trim to top 2000 only
Xcontext_vectorized = xvectorizer.transform(Xcontext_train)
yquestion_vectorized = ylblencoder.transform(yquestion_train)

Xquestion_vectorized = xvectorizer.transform(Xquestion_train)
yanswer_vectorized = ylblencoder.transform(yanswer_train)

In [0]:
Xcontext_test_vectorized= xvectorizer.transform(Xcontext_test)
yquestion_test_vectorized= xvectorizer.transform(yquestion_test)

Xquestion_test_vectorized = xvectorizer.transform(Xquestion_test)
yanswer_test_vectorized = ylblencoder.transform(yanswer_test)

In [0]:
from sklearn import svm

# SVM model 1
try:
  context2question = svm.SVC()    
  context2question.fit(Xcontext_vectorized,yquestion_vectorized)
except RuntimeError as e:
  print(e)

print('Model context2question', context2question)

In [0]:
try: 
  question2answer = svm.SVC(gamma=0.025, C=10)    
  question2answer.fit(Xquestion_vectorized,yanswer_vectorized)
except RuntimeError as e:
  print(e)

print('Model question2answer', question2answer)

In [0]:
train_acc = context2question.score(Xcontext_vectorized,yquestion_vectorized)
test_acc = context2question.score(Xcontext_vectorized,yquestion_vectorized)